In [1]:
import pandas as pd
import yaml
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
with open("../config.yaml", "r") as file:
    paths = yaml.safe_load(file)

print(paths['data']['clean']['file1'])
print(paths['data']['clean']['file2'])
print(paths['data']['clean']['file3'])
print(paths['data']['clean']['file4'])
print(paths['data']['clean']['file5'])
print(paths['data']['clean']['file6'])
print(paths['data']['clean']['file7'])

../data/clean/earthquake_data_clean.csv
../data/clean/inventory_clean_redundant.csv
../data/clean/value_1997-2020_clean.csv
../data/clean/value_2019-2024_clean.csv
../data/clean/values_joined_clean.csv
../data/clean/values_pivotedback.csv
../data/clean/combined_df_value+inventory.csv


In [3]:
# load files

df1 = pd.read_csv(paths['data']['clean']['file7']) #, usecols=[0, 1, 2, 3, 4])
df2 = pd.read_csv(paths['data']['clean']['file6'])
df3 = pd.read_csv(paths['data']['clean']['file2'])
df4 = pd.read_csv(paths['data']['clean']['file4'])


Hypothesis testing value affected city vs benchmark

In [4]:
# preliminary view

df1.head()

,index,region,year,value,total,total_occupied,total_not_occupied,oppucation_ratio
0,0,'s-Gravenhage (gemeente),2012,207.0,241376.0,219180.0,22196.0,0.908044
1,1,'s-Gravenhage (gemeente),2013,199.0,244880.0,222196.0,22684.0,0.907367
2,2,'s-Gravenhage (gemeente),2014,188.0,249994.0,225202.0,24792.0,0.900830
3,3,'s-Gravenhage (gemeente),2015,184.0,251374.0,227464.0,23910.0,0.904883
4,4,'s-Gravenhage (gemeente),2016,188.0,253210.0,230398.0,22812.0,0.909909


In [5]:
df1['region'].unique()

array(["'s-Gravenhage (gemeente)", "'s-Hertogenbosch", 'Aa en Hunze',
       'Aalburg', 'Aalsmeer', 'Aalten', 'Achtkarspelen', 'Alblasserdam',
       'Albrandswaard', 'Alkmaar', 'Almelo', 'Almere',
       'Alphen aan den Rijn', 'Alphen-Chaam', 'Altena', 'Ameland',
       'Amersfoort', 'Amstelveen', 'Amsterdam', 'Apeldoorn', 'Appingedam',
       'Arnhem', 'Assen', 'Asten', 'Baarle-Nassau', 'Baarn',
       'Barendrecht', 'Barneveld', 'Bedum', 'Beek (L.)', 'Beekdaelen',
       'Beemster', 'Beesel', 'Bellingwedde', 'Berg en Dal', 'Bergambacht',
       'Bergeijk', 'Bergen (L.)', 'Bergen (NH.)', 'Bergen op Zoom',
       'Berkelland', 'Bernheze', 'Bernisse', 'Best', 'Beuningen',
       'Beverwijk', 'Binnenmaas', 'Bladel', 'Blaricum', 'Bloemendaal',
       'Boarnsterhim', 'Bodegraven-Reeuwijk', 'Boekel', 'Borger-Odoorn',
       'Borne', 'Borsele', 'Boskoop', 'Boxmeer', 'Boxtel', 'Breda',
       'Brielle', 'Bronckhorst', 'Brummen', 'Brunssum', 'Bunnik',
       'Bunschoten', 'Buren', 'Bussum', '

In [6]:
search_string = 'PV'
matching_rows = df1[df1['region'].str.contains(search_string, case=False, na=False)]

matching_rows['region'].unique()

array(['Drenthe (PV)', 'Flevoland (PV)', 'Friesland (PV)',
       'Gelderland (PV)', 'Groningen (PV)', 'Limburg (PV)',
       'Noord-Brabant (PV)', 'Noord-Holland (PV)', 'Overijssel (PV)',
       'Utrecht (PV)', 'Zeeland (PV)', 'Zuid-Holland (PV)'], dtype=object)

In [7]:
# output2 = df1[(df1['region'] == 'Leeuwarden') | (df1['region'] == 'Appingedam')]
# output2 = output2.dropna()
# output2.to_csv('output2.csv', index=False)

In [8]:
df_target_city = df1[df1['region'] == 'Appingedam']
df_benchmark_city = df1[df1['region'] == 'Franekeradeel']
df_target_region = df1[df1['region'] == 'Groningen (PV)']
df_benchmark_region = df1[df1['region'] == 'Friesland (PV)']

# drop NaNs
df_target_city = df_target_city.dropna()
df_benchmark_city = df_benchmark_city.dropna()
df_target_region = df_target_region.dropna()
df_benchmark_region = df_benchmark_region.dropna()

In [9]:
# #### DECOMISSIONED ###

# # Weighted two-sample T-test - Target city vs Benchmark city

# # Extract values and weights for both regions
# values_target = df_target_city['value']
# weights_target = df_target_city['total']

# values_benchmark = df_benchmark_city['value']
# weights_benchmark = df_benchmark_city['total']

# # Function to compute the weighted mean
# def weighted_mean(values, weights):
#     return np.sum(values * weights) / np.sum(weights)

# # Function to compute the weighted variance
# def weighted_variance(values, weights):
#     mean = weighted_mean(values, weights)
#     variance = np.sum(weights * (values - mean) ** 2) / np.sum(weights)
#     return variance

# # Calculate the weighted means
# mean_target = weighted_mean(values_target, weights_target)
# mean_benchmark = weighted_mean(values_benchmark, weights_benchmark)

# # Calculate the weighted variances
# var_target = weighted_variance(values_target, weights_target)
# var_benchmark = weighted_variance(values_benchmark, weights_benchmark)

# # Calculate the number of samples (n)
# n_target = len(values_target)
# n_benchmark = len(values_benchmark)

# # Calculate the t-statistic
# t_statistic = (mean_target - mean_benchmark) / np.sqrt((var_target / n_target) + (var_benchmark / n_benchmark))

# # Degrees of freedom for weighted t-test (approximation using Welch-Satterthwaite equation)
# df_numerator = ((var_target / n_target) + (var_benchmark / n_benchmark)) ** 2
# df_denominator = ((var_target / n_target) ** 2 / (n_target - 1)) + ((var_benchmark / n_benchmark) ** 2 / (n_benchmark - 1))
# degrees_of_freedom = df_numerator / df_denominator

# # Calculate the p-value for a one-sided t-test (to test if target is lower than benchmark)
# p_value = stats.t.cdf(t_statistic, df=degrees_of_freedom)

# # Output the results
# print(f"t-statistic: {t_statistic}")
# print(f"p-value: {p_value}")

# alpha = 0.05
# if p_value < alpha:
#     print("Reject the null hypothesis: target city has a lower value than benchmark city.")
# else:
#     print("Fail to reject the null hypothesis: No evidence that target city has a lower value than benchmark city.")

Weighted two-sample T-test - Target region vs Benchmark region

In [10]:
# #### DECOMISSIONED ###

# # Weighted two-sample T-test - Target region vs Benchmark region

# # Extract values and weights for both regions
# values_target = df_target_region['value']
# weights_target = df_target_region['total']

# values_benchmark = df_benchmark_region['value']
# weights_benchmark = df_benchmark_region['total']

# # Function to compute the weighted mean
# def weighted_mean(values, weights):
#     return np.sum(values * weights) / np.sum(weights)

# # Function to compute the weighted variance
# def weighted_variance(values, weights):
#     mean = weighted_mean(values, weights)
#     variance = np.sum(weights * (values - mean) ** 2) / np.sum(weights)
#     return variance

# # Calculate the weighted means
# mean_target = weighted_mean(values_target, weights_target)
# mean_benchmark = weighted_mean(values_benchmark, weights_benchmark)

# # Calculate the weighted variances
# var_target = weighted_variance(values_target, weights_target)
# var_benchmark = weighted_variance(values_benchmark, weights_benchmark)

# # Calculate the number of samples (n)
# n_target = len(values_target)
# n_benchmark = len(values_benchmark)

# # Calculate the t-statistic
# t_statistic = (mean_target - mean_benchmark) / np.sqrt((var_target / n_target) + (var_benchmark / n_benchmark))

# # Degrees of freedom for weighted t-test (approximation using Welch-Satterthwaite equation)
# df_numerator = ((var_target / n_target) + (var_benchmark / n_benchmark)) ** 2
# df_denominator = ((var_target / n_target) ** 2 / (n_target - 1)) + ((var_benchmark / n_benchmark) ** 2 / (n_benchmark - 1))
# degrees_of_freedom = df_numerator / df_denominator

# # Calculate the p-value for a one-sided t-test (to test if target is lower than benchmark)
# p_value = stats.t.cdf(t_statistic, df=degrees_of_freedom)

# # Output the results
# print(f"t-statistic: {t_statistic}")
# print(f"p-value: {p_value}")

# alpha = 0.05
# if p_value < alpha:
#     print("Reject the null hypothesis: target region has a lower value than benchmark region.")
# else:
#     print("Fail to reject the null hypothesis: No evidence that target region has a lower value than benchmark region.")

In [11]:

# Filter data for "Target city" and "Benchmark"
df_target_city = df1[df1['region'] == 'Appingedam']
df_benchmark_city = df1[df1['region'] == 'Franekeradeel']

# Extract the occupation_ratio for both regions
occupation_targetcity = df_target_city['oppucation_ratio'].dropna()
occupation_benchmarkcity = df_benchmark_city['oppucation_ratio'].dropna()

# Perform Welch's t-test for unequal variances
t_statistic, p_value = stats.ttest_ind(occupation_targetcity, occupation_benchmarkcity, equal_var=False, alternative='less')

# Output the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Check if we can reject the null hypothesis at a significance level of 0.05
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: target city has a lower occupation ratio than benchmark city.")
else:
    print("Fail to reject the null hypothesis: No evidence that target city has a lower occupation ratio than benchmark city.")


t-statistic: 3.4702883077753923
p-value: 0.998904479527389
Fail to reject the null hypothesis: No evidence that target city has a lower occupation ratio than benchmark city.


In [12]:

# Filter data for "Target region" and "Benchmark"
df_target_region = df1[df1['region'] == 'Groningen (PV)']
df_benchmark_region = df1[df1['region'] == 'Friesland (PV)']

# Extract the occupation_ratio for both regions
occupation_targetregion = df_target_region['oppucation_ratio'].dropna()
occupation_benchmarkregion = df_benchmark_region['oppucation_ratio'].dropna()

# Perform Welch's t-test for unequal variances
t_statistic, p_value = stats.ttest_ind(occupation_targetregion, occupation_benchmarkregion, equal_var=False, alternative='less')

# Output the results
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Check if we can reject the null hypothesis at a significance level of 0.05
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis: target region has a lower occupation ratio than benchmark region.")
else:
    print("Fail to reject the null hypothesis: No evidence that target region has a lower occupation ratio than benchmark region.")

t-statistic: 0.7717475870976036
p-value: 0.7752359957977669
Fail to reject the null hypothesis: No evidence that target region has a lower occupation ratio than benchmark region.


In [13]:
output2 = df2[(df2['region'] == 'Franekeradeel') | (df2['region'] == 'Appingedam')]

In [14]:
# output2.to_csv('output3.csv', index=False)

In [15]:
output2

,region,year,value
952,Appingedam,1997,52.0
953,Appingedam,1998,52.0
954,Appingedam,1999,52.0
955,Appingedam,2000,53.0
956,Appingedam,2001,74.0
957,Appingedam,2002,74.0
958,Appingedam,2003,76.0
959,Appingedam,2004,77.0
960,Appingedam,2005,117.0
961,Appingedam,2006,118.0


In [25]:
df2['region'].unique()

array(["'s-Graveland", "'s-Gravendeel", "'s-Gravenhage (gemeente)",
       "'s-Gravenzande", "'s-Hertogenbosch", 'Aa en Hunze', 'Aalburg',
       'Aalsmeer', 'Aalten', 'Abcoude', 'Achtkarspelen', 'Akersloot',
       'Alblasserdam', 'Albrandswaard', 'Alkemade', 'Alkmaar', 'Almelo',
       'Almere', 'Alphen aan den Rijn', 'Alphen-Chaam', 'Altena',
       'Ambt Delden', 'Ambt Montfort', 'Ameland', 'Amerongen',
       'Amersfoort', 'Ammerzoden', 'Amstelveen', 'Amsterdam', 'Andijk',
       'Angerlo', 'Anloo', 'Anna Paulowna', 'Apeldoorn', 'Appingedam',
       'Arcen en Velden', 'Arnhem', 'Assen', 'Asten', 'Avereest', 'Axel',
       'Baarle-Nassau', 'Baarn', 'Barendrecht', 'Barneveld', 'Bathmen',
       'Bedum', 'Beek (L.)', 'Beekdaelen', 'Beemster', 'Beesel', 'Beilen',
       'Belfeld', 'Bellingwedde', 'Bemmel', 'Bennebroek', 'Berg en Dal',
       'Bergambacht', 'Bergeijk', 'Bergen (L.)', 'Bergen (NH.)',
       'Bergen op Zoom', 'Bergeyk', 'Bergh', 'Bergschenhoek',
       'Berkel en Rodenri

In [16]:
######### USE THIS  ########

# Separate data for each region
target = df2[df2['region'] == 'Appingedam'].set_index('year')['value']
bm = df2[df2['region'] == 'Franekeradeel'].set_index('year')['value']

# Ensure we're comparing the same years
common_years = target.index.intersection(bm.index)
target = target[common_years]
bm = bm[common_years]

# Calculate the differences (target - bm)
differences = target - bm

# Perform one-tailed paired t-test
t_statistic, p_value = stats.ttest_rel(target, bm, alternative='less')

print(f"One-tailed paired t-test statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("There is significant evidence that prices in target are lower than in BM.")
else:
    print("There is not enough evidence to conclude that prices in target are significantly lower than in BM.")

# Additional descriptive statistics
print("\nDescriptive Statistics:")
print("Target:")
print(f"Mean: {np.mean(target)}")
print(f"Standard Deviation: {np.std(target, ddof=1)}")
print("\nBM:")
print(f"Mean: {np.mean(bm)}")
print(f"Standard Deviation: {np.std(bm, ddof=1)}")

# Calculate and print the mean difference
mean_diff = np.mean(differences)
print(f"\nMean difference (target - bm): {mean_diff}")

# Count how many years target was lower
lower_count = np.sum(differences < 0)
total_years = len(differences)
print(f"\ntarget had lower prices in {lower_count} out of {total_years} years.")

One-tailed paired t-test statistic: -8.032632968331516
p-value: 6.219933143574666e-09
There is significant evidence that prices in target are lower than in BM.

Descriptive Statistics:
Target:
Mean: 126.42857142857143
Standard Deviation: 46.631279099722214

BM:
Mean: 140.21428571428572
Standard Deviation: 52.36648429337869

Mean difference (target - bm): -13.785714285714286

target had lower prices in 24 out of 28 years.


In [26]:


# Separate data for each region
target = df2[df2['region'] == 'Groningen (PV)'].set_index('year')['value']
bm = df2[df2['region'] == 'Friesland (PV)'].set_index('year')['value']

# Ensure we're comparing the same years
common_years = target.index.intersection(bm.index)
target = target[common_years]
bm = bm[common_years]

# Calculate the differences (target - bm)
differences = target - bm

# Perform one-tailed paired t-test
t_statistic, p_value = stats.ttest_rel(target, bm, alternative='less')

print(f"One-tailed paired t-test statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("There is significant evidence that prices in target are lower than in BM.")
else:
    print("There is not enough evidence to conclude that prices in target are significantly lower than in BM.")

# Additional descriptive statistics
print("\nDescriptive Statistics:")
print("Target:")
print(f"Mean: {np.mean(target)}")
print(f"Standard Deviation: {np.std(target, ddof=1)}")
print("\nBM:")
print(f"Mean: {np.mean(bm)}")
print(f"Standard Deviation: {np.std(bm, ddof=1)}")

# Calculate and print the mean difference
mean_diff = np.mean(differences)
print(f"\nMean difference (target - bm): {mean_diff}")

# Count how many years target was lower
lower_count = np.sum(differences < 0)
total_years = len(differences)
print(f"\ntarget had lower prices in {lower_count} out of {total_years} years.")

One-tailed paired t-test statistic: -11.235333258139937
p-value: 5.512048154649767e-12
There is significant evidence that prices in target are lower than in BM.

Descriptive Statistics:
Target:
Mean: 150.53571428571428
Standard Deviation: 59.50442382736308

BM:
Mean: 162.07142857142858
Standard Deviation: 62.26641004671636

Mean difference (target - bm): -11.535714285714286

target had lower prices in 26 out of 28 years.


In [39]:
output3 = df3[(df3['region'] == 'Groningen (PV)') | (df3['region'] == 'Fryslân (PV)')]
output3

,region,period,total,total_occupied,total_not_occupied,oppucation_ratio
12,Groningen (PV),01/01/2012,265332.0,250946.0,14386.0,0.945781
13,Groningen (PV),01/01/2013,267466.0,253549.0,13917.0,0.947967
14,Groningen (PV),01/01/2014,271087.0,257294.0,13793.0,0.949120
15,Groningen (PV),01/01/2015,272774.0,259458.0,13316.0,0.951183
16,Groningen (PV),01/01/2016,274719.0,261641.0,13078.0,0.952395
17,Groningen (PV),01/01/2017,276728.0,263918.0,12810.0,0.953709
18,Groningen (PV),01/01/2018,277113.0,264623.0,12490.0,0.954928
19,Groningen (PV),01/01/2019,279662.0,266736.0,12926.0,0.953780
20,Groningen (PV),01/01/2020,282047.0,269174.0,12873.0,0.954359
21,Groningen (PV),01/01/2021,285012.0,272092.0,12920.0,0.954669


In [35]:
# output3.to_csv('output4.csv', index=False)
df3

,region,period,total,total_occupied,total_not_occupied,oppucation_ratio
0,Nederland,01/01/2012,7386743.0,6975208.0,411535.0,0.944287
1,Nederland,01/01/2013,7449298.0,7055105.0,394193.0,0.947083
2,Nederland,01/01/2014,7535315.0,7143163.0,392152.0,0.947958
3,Nederland,01/01/2015,7587964.0,7211229.0,376735.0,0.950351
4,Nederland,01/01/2016,7641323.0,7276184.0,365139.0,0.952215
...,...,...,...,...,...,...
5444,Zwolle,01/01/2020,58492.0,56910.0,1582.0,0.972954
5445,Zwolle,01/01/2021,59491.0,57851.0,1640.0,0.972433
5446,Zwolle,01/01/2022,60228.0,58516.0,1712.0,0.971575
5447,Zwolle,01/01/2023,60475.0,58864.0,1611.0,0.973361


In [36]:

# Separate data for each region
target_pv = df3[df3['region'] == 'Groningen (PV)'].set_index('period')['oppucation_ratio']
bm_pv = df3[df3['region'] == 'Fryslân (PV)'].set_index('period')['oppucation_ratio']

# Ensure we're comparing the same periods
common_periods = target_pv.index.intersection(bm_pv.index)
target_pv = target_pv[common_periods]
bm_pv = bm_pv[common_periods]

# Calculate the differences (target_pv - bm_pv)
differences = target_pv - bm_pv

# Perform one-tailed paired t-test
t_statistic, p_value = stats.ttest_rel(target_pv, bm_pv, alternative='less')

print(f"One-tailed paired t-test statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("There is significant evidence that the occupation ratio in target_pv (PV) is lower than in bm_pv (PV).")
else:
    print("There is not enough evidence to conclude that the occupation ratio in target_pv (PV) is significantly lower than in bm_pv (PV).")

# Additional descriptive statistics
print("\nDescriptive Statistics:")
print("target_pv (PV):")
print(f"Mean occupation ratio: {np.mean(target_pv):.6f}")
print(f"Standard Deviation: {np.std(target_pv, ddof=1):.6f}")
print("\nbm_pv (PV):")
print(f"Mean occupation ratio: {np.mean(bm_pv):.6f}")
print(f"Standard Deviation: {np.std(bm_pv, ddof=1):.6f}")

# Calculate and print the mean difference
mean_diff = np.mean(differences)
print(f"\nMean difference (target_pv - bm_pv): {mean_diff:.6f}")

# Count how many periods target_pv was lower
lower_count = np.sum(differences < 0)
total_periods = len(differences)
print(f"\ntarget_pv had a lower occupation ratio in {lower_count} out of {total_periods} periods.")

One-tailed paired t-test statistic: 1.686263013300175
p-value: 0.9400668291828441
There is not enough evidence to conclude that the occupation ratio in target_pv (PV) is significantly lower than in bm_pv (PV).

Descriptive Statistics:
target_pv (PV):
Mean occupation ratio: 0.951890
Standard Deviation: 0.002958

bm_pv (PV):
Mean occupation ratio: 0.950724
Standard Deviation: 0.004313

Mean difference (target_pv - bm_pv): 0.001165

target_pv had a lower occupation ratio in 2 out of 12 periods.


In [42]:
print(target_pv)
print(bm_pv)

period
01/01/2012    0.946790
01/01/2013    0.938462
01/01/2014    0.944863
01/01/2015    0.952821
01/01/2016    0.955960
01/01/2017    0.957281
01/01/2018    0.951561
01/01/2019    0.931863
01/01/2020    0.933300
01/01/2021    0.938843
01/01/2022    0.937500
01/01/2023    0.937092
Name: oppucation_ratio, dtype: float64
period
01/01/2012    0.932604
01/01/2013    0.937102
01/01/2014    0.935226
01/01/2015    0.938765
01/01/2016    0.939270
01/01/2017    0.939560
01/01/2018    0.939140
01/01/2019    0.932769
01/01/2020    0.919934
01/01/2021    0.922760
01/01/2022    0.919570
01/01/2023    0.911312
Name: oppucation_ratio, dtype: float64


In [37]:

# Separate data for each region
target_pv = df3[df3['region'] == 'Appingedam'].set_index('period')['oppucation_ratio']
bm_pv = df3[df3['region'] == 'Franekeradeel'].set_index('period')['oppucation_ratio']

# Ensure we're comparing the same periods
common_periods = target_pv.index.intersection(bm_pv.index)
target_pv = target_pv[common_periods]
bm_pv = bm_pv[common_periods]

# Calculate the differences (target_pv - bm_pv)
differences = target_pv - bm_pv

# Perform one-tailed paired t-test
t_statistic, p_value = stats.ttest_rel(target_pv, bm_pv, alternative='less')

print(f"One-tailed paired t-test statistic: {t_statistic}")
print(f"p-value: {p_value}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("There is significant evidence that the occupation ratio in target_pv (PV) is lower than in bm_pv (PV).")
else:
    print("There is not enough evidence to conclude that the occupation ratio in target_pv (PV) is significantly lower than in bm_pv (PV).")

# Additional descriptive statistics
print("\nDescriptive Statistics:")
print("target_pv (PV):")
print(f"Mean occupation ratio: {np.mean(target_pv):.6f}")
print(f"Standard Deviation: {np.std(target_pv, ddof=1):.6f}")
print("\nbm_pv (PV):")
print(f"Mean occupation ratio: {np.mean(bm_pv):.6f}")
print(f"Standard Deviation: {np.std(bm_pv, ddof=1):.6f}")

# Calculate and print the mean difference
mean_diff = np.mean(differences)
print(f"\nMean difference (target_pv - bm_pv): {mean_diff:.6f}")

# Count how many periods target_pv was lower
lower_count = np.sum(differences < 0)
total_periods = len(differences)
print(f"\ntarget_pv had a lower occupation ratio in {lower_count} out of {total_periods} periods.")

One-tailed paired t-test statistic: 6.317451935602701
p-value: 0.999971514483089
There is not enough evidence to conclude that the occupation ratio in target_pv (PV) is significantly lower than in bm_pv (PV).

Descriptive Statistics:
target_pv (PV):
Mean occupation ratio: 0.943861
Standard Deviation: 0.008902

bm_pv (PV):
Mean occupation ratio: 0.930668
Standard Deviation: 0.009706

Mean difference (target_pv - bm_pv): 0.013194

target_pv had a lower occupation ratio in 1 out of 12 periods.
